# Liver Model Construction: Glycolysis Gluconeogenesis & Lactate Fermentation

## Setup workflow

### Import packages

In [1]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sy
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass import MassMetabolite, MassModel, MassReaction
from mass.io.json import save_json_model as save_json_mass_model

print("MASSpy version: {0}".format(mass.__version__))

MASSpy version: 0.1.1


## Set paths and constants

### Directory paths

model_dir = os.path.abspath("../models")
maps_dir = os.path.abspath("../maps")

In [2]:
# **changed the path depending on the file on where the files are located**
model_dir = os.path.abspath("/Users/RiyaMathur/MASSliver/models")
maps_dir = os.path.abspath("/Users/RiyaMathur/MASSliver/models/maps")
data_dir = os.path.abspath("/Users/RiyaMathur/MASSliver/data/interim")

# Create MASS model from COBRA model

### Load COBRA model

In [3]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
recon=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
core_model

Name,None
Memory address,0x07f8b14418af0
Number of metabolites,178
Number of reactions,187
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


## Create MASS model

In [4]:
glycogenolysis = MassModel("Glycogenolysis", array_type='DataFrame', dtype=np.int64)

### Define reactions

In [ ]:
# glycolysis_map = escher.Builder(model=core_model,  map_json="Glycolysis, gluconeogenesis lactate fermentation 2.json",
#                                 highlight_missing=True) 
# glycolysis_map

In [5]:
reaction_list = ['HEX1',
                'PGI',
                'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L',
                'PYRt2m', 
              #  'SUCR', This is a sucrose metabolism reaction... 
                'G6PDH2r',
                'PGMT',
                'G6Pter',
                'GLCter',
                'TKT2',
                'PGCD', 
                '3SPYRSP', 
                'ALATA_L', 
                '2AMACHYD', 
               # 'GLCt1', This is a surcose metabolism reaction...
                'PEPCK_re',
                'MDH',
                'MALtm', 
                'EX_glc__D_c',
                'EX_pyr_c',
                'EX_lac__L_c',
                'G6Pter',
                'G6PPer',
                'GLCter']



new = ['EX_glc__D_e', 'GLCt1'] #from RECONN


new_reaction_list = []
for item in reaction_list: 
    item = core_model.reactions.get_by_id(item)
    new_reaction_list.append(item) 

new_reaction_list
for i in new: 
    i = recon.reactions.get_by_id(i)
    new_reaction_list.append(i) 

# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])
for rid in new:
    reaction = recon.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])  
new_reaction_list

metabolite = core_model.metabolites.get_by_id('g6p_c')

SK_g6p_c = glycogenolysis.add_boundary(
    metabolite= 'g6p_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_g6p_c)]) 
new_reaction_list.append(SK_g6p_c)

print(new_reaction_list)

Ignoring reaction 'G6Pter' since it already exists.
Ignoring reaction 'GLCter' since it already exists.
Ignoring reaction 'SK_g6p_c' since it already exists.


[<Reaction HEX1 at 0x7f8b1d21ca00>, <Reaction PGI at 0x7f8b1d21c8e0>, <Reaction FBP at 0x7f8b1d232fa0>, <Reaction PFK at 0x7f8b1d21c040>, <Reaction FBA at 0x7f8b1d21c430>, <Reaction TPI at 0x7f8b1d1fbee0>, <Reaction GAPD at 0x7f8b1d21c400>, <Reaction PGK at 0x7f8b1d229700>, <Reaction PGM at 0x7f8b1d229640>, <Reaction ENO at 0x7f8b1d37fa30>, <Reaction PEPtm at 0x7f8b1d341160>, <Reaction PEPCKm at 0x7f8b1d232100>, <Reaction PYK at 0x7f8b1d37f7c0>, <Reaction PCm at 0x7f8b1d2328e0>, <Reaction LDH_L at 0x7f8b1d3d9dc0>, <Reaction PYRt2m at 0x7f8b1d3412b0>, <Reaction G6PDH2r at 0x7f8b1d3d9af0>, <Reaction PGMT at 0x7f8b1d235af0>, <Reaction G6Pter at 0x7f8b1d232580>, <Reaction GLCter at 0x7f8b1d232c10>, <Reaction TKT2 at 0x7f8b1d215460>, <Reaction PGCD at 0x7f8b1d3a85e0>, <Reaction 3SPYRSP at 0x7f8b1d3a8280>, <Reaction ALATA_L at 0x7f8b1d3a2f70>, <Reaction 2AMACHYD at 0x7f8b1d37c4c0>, <Reaction PEPCK_re at 0x7f8b1d235bb0>, <Reaction MDH at 0x7f8b1d2356d0>, <Reaction MALtm at 0x7f8b1d235970>, <R

### View extracted network

In [6]:
escher_builder = escher.Builder(
    model=glycogenolysis,
    map_json=os.path.join(
        model_dir, ".".join((
            glycogenolysis.id, "map", "json"))
    ),
    highlight_missing=True)

escher_builder

NameError: name 'glycolysis' is not defined

## Parameterize MASS model

# Export MASS model

In [7]:
save_json_mass_model(
    mass_model=glycogenolysis,
    filename=os.path.join(model_dir, glycogenolysis.id + ".json"))